## 解答 04_08
> * 將 下列圖片 彩色載入
> * 垂直 rotate 90 度, show 出結果
> * 再 3x3 高斯濾波器, sigmaX=0.8, sigmaY=0.8, show 出結果<br>
> * 再 sobelX, sobelY 後用(0.5, 0.5)的 權重加起來, show 出結果<br>
> * 再輪廓偵測外框 (contours) 將找到的輪廓填滿畫在原圖上, show 出結果
> * 將上圖中的物件中心用圓圈標示出來, 並將面積標示於中心旁邊

### 必須將上列個各處理後的圖 imshow 出來

In [4]:
import cv2
import numpy as np

img = cv2.imread('../image/contour.png')
cv2.imshow('original', img)
cv2.waitKey(0)

# -------- 垂直 rotate 90 度 ------------------------
imgFlip = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
cv2.imshow('Rotate', imgFlip)
cv2.waitKey(0)


# ----- 3*3 高斯濾波器 -----------------------------
imgGs =cv2.GaussianBlur(imgFlip, (3, 3), .8, .8)
cv2.imshow('GaussianBlur', imgGs)
cv2.waitKey(0)


# ------- 將前一處理後圖片取 SobelX, SobelY and addWeight(0.5, 0.5)----------
sobelx = cv2.Sobel(imgGs, cv2.CV_64F, 1, 0)
sobely = cv2.Sobel(imgGs, cv2.CV_64F, 0, 1)

sobelx = cv2.convertScaleAbs(sobelx)   # 轉回 uint8 
sobely = cv2.convertScaleAbs(sobely)  

sobelxy =  cv2.addWeighted(sobelx, 1, sobely, 1, 0)
cv2.imshow('sobelxy', sobelxy)  
cv2.waitKey(0)


# ------- 輪廓偵測 (contours) 將找到的輪廓畫在原圖上 ----------
sobelxy_g = cv2.cvtColor(sobelxy, cv2.COLOR_BGR2GRAY)
cnts, hierarchy = cv2.findContours(sobelxy_g, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
mask = cv2.drawContours(sobelxy, cnts, -1, (0,0,255), -1)
cv2.imshow('contour', mask)
cv2.waitKey(0)


# ------- 將上圖中的物件中心用圓圈標示出來, 並將面積標示於中心旁邊 ---------
for i in range(len(cnts)):
    M = cv2.moments(cnts[i])
    cx = int(M['m10'] / M['m00'])  # 中心點 x 座標
    cy = int(M['m01'] / M['m00'])  # 中心點 y 座標
    
    area = cv2.contourArea(cnts[i])                                        # 面積
    round_len = cv2.arcLength(cnts[i], True)                     # 週長
    cv2.circle(mask, (cx,cy), 3, (0,255,0), -2)
    cv2.putText(mask, f'area = {area}', (cx-60,cy-10), cv2.FONT_HERSHEY_SIMPLEX,  .6, (0, 255, 0), 2, cv2.LINE_AA)
    print(f'輪廓 {i} 的中心點 ({cx}, {cy}),\t面積 : {area:10,.2f},\t週長 :{round_len:9,.2f}')
    # temp=np.zeros(im.shape, np.uint8)
cv2.imshow('mask', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1) 

輪廓 0 的中心點 (244, 526),	面積 :  45,035.00,	週長 :   793.49
輪廓 1 的中心點 (384, 353),	面積 :  11,693.50,	週長 :   677.71
輪廓 2 的中心點 (104, 254),	面積 :  36,794.00,	週長 :   797.13
輪廓 3 的中心點 (349, 144),	面積 :  25,284.00,	週長 :   650.00


-1